In [2]:
from pathlib import Path
import sys, importlib.util

def add_project_root(marker="src", max_up=6):
    """현재 CWD에서 위로 올라가며 'src' 폴더가 보이는 지점을 sys.path에 추가"""
    p = Path.cwd()
    for _ in range(max_up):
        if (p/marker).exists():
            p_str = str(p)
            if p_str not in sys.path:
                sys.path.insert(0, p_str)  # 최우선
            return p
        p = p.parent
    raise RuntimeError(f"'{marker}' not found within {max_up} parents from {Path.cwd()}")

root = add_project_root()  # 보통 /home/.../Projects/minervini
print("USING ROOT:", root)
print("ROOT/src exists?:", (root/"src").exists())
print("find_spec('src'):", importlib.util.find_spec("src"))

USING ROOT: /home/brighter87/Projects/minervini
ROOT/src exists?: True
find_spec('src'): ModuleSpec(name='src', loader=None, submodule_search_locations=_NamespacePath(['/home/brighter87/Projects/minervini/src']))


In [3]:
import mplfinance as mpf
from src.analysis.prepare import get_ohlc_ma_rs_for_analysis

df = get_ohlc_ma_rs_for_analysis()

In [4]:
from src.analysis.screen_minervini import screen_minervini

result_df = screen_minervini(df, rs_col="RS_200_pct")

In [5]:
len(result_df)

809

In [10]:
result_df.loc[result_df["v"] > 10000000]

,T,v,vw,o,c,h,l,t,n,date,...,RS_200,RS_50_pct,RS_150_pct,RS_200_pct,RS_pct_min,RS_pct_mean,52w_high,52w_low,is_ma200_up,meets_all
2560879,ONDS,36920819.0,7.6393,7.695,7.660,7.8900,7.3600,1758916800000,86427.0,2025-09-26,...,8.417521,0.992788,0.997085,0.997614,0.992788,0.995829,7.88,0.6710,True,True
2918387,RGTI,100168691.0,31.9085,32.155,31.180,33.9800,30.7400,1758916800000,655788.0,2025-09-26,...,6.288680,0.967641,0.977259,0.995824,0.967641,0.980241,32.10,0.7402,True,True
2823513,QBTS,60651933.0,26.8330,26.350,26.760,28.5000,26.0200,1758916800000,291508.0,2025-09-26,...,4.671895,0.897189,0.988144,0.993438,0.897189,0.959590,27.72,0.8931,True,True
2542616,OKLO,24836639.0,112.0527,117.630,110.530,119.9600,108.5400,1758916800000,345938.0,2025-09-26,...,4.456514,0.952848,0.976288,0.992842,0.952848,0.973992,142.65,8.0900,True,True
2353786,NB,11179426.0,6.8665,7.310,6.820,7.4000,6.5600,1758916800000,43498.0,2025-09-26,...,3.543994,0.982618,0.980952,0.989063,0.980952,0.984211,7.31,1.3000,True,True
2572498,OPEN,168062513.0,8.6715,8.800,8.810,8.9800,8.3700,1758916800000,337615.0,2025-09-26,...,3.537616,0.997966,0.997473,0.988865,0.988865,0.994768,10.52,0.5122,True,True
2270041,MP,18999946.0,71.3465,77.000,68.630,77.4700,68.5700,1758916800000,229016.0,2025-09-26,...,3.074430,0.730399,0.975899,0.981905,0.730399,0.896068,77.13,15.6000,True,True
1186970,EOSE,12679248.0,10.0985,10.500,10.120,10.6080,9.8300,1758916800000,66401.0,2025-09-26,...,2.785050,0.973373,0.959961,0.977530,0.959961,0.970288,10.65,2.1500,True,True
2712138,PL,10400940.0,12.1618,11.860,12.200,12.3300,11.7600,1758916800000,61305.0,2025-09-26,...,2.590741,0.969675,0.966764,0.974349,0.966764,0.970262,12.20,2.1000,True,True
1659141,HOOD,25721873.0,121.9953,124.180,121.780,125.0600,120.4600,1758916800000,254999.0,2025-09-26,...,2.581725,0.749445,0.964431,0.973951,0.749445,0.895943,126.80,22.3300,True,True
